In [9]:
import pandas as pd
import folium
import re
import googlemaps
import ast
import time

In [2]:
df = pd.read_csv("youtube_videos.csv")

In [3]:
with open("youtube_api_key.txt","r") as file:
    api_key = file.read().strip()

In [4]:
import requests
api_response = requests.get("https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8")
api_response.text

'<!DOCTYPE html><html itemscope="" itemtype="http://schema.org/Place" lang="en"> <head>  <link href="/maps/_/js/k=maps.m.en.FbL2lLnzRu4.es5.O/m=sc2,per,mo,lp,ep,ti,ds,stx,dwi,enr,pwd,dw,ppl,log,std,b/am=BqEMDUg/rt=j/d=1/rs=ACT90oEDH7QqeRj0UGSjB5r3gdwJVg-_HA?wli=m.2catmk6Grj4.loadSv.O%3A%3B" as="script" rel="preload" type="application/javascript" nonce="KHsOfe80L8d30POPvd6Lgg">   <link href="/maps/preview/opensearch.xml?hl=en" title="Google Maps" rel="search" type="application/opensearchdescription+xml"> <title>  Google Maps  </title> <meta content=" Find local businesses, view maps and get driving directions in Google Maps. " name="Description"> <meta content="Anm+hhtuh7NJguqSnXHEAIqqMaV+GXCks8WYXHJKF7l6AeYMj+wO+fi9OdDqFnJTg9t0492DykVxx4jpvFbxnA8AAABseyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IlByaXZhY3lTYW5kYm94QWRzQVBJcyIsImV4cGlyeSI6MTY5NTE2Nzk5OSwiaXNTdWJkb21haW4iOnRydWV9" http-equiv="origin-trial">  <meta content="initial-scale=1.0, maximum-scale=1.0, minimum-scal

In [5]:
api_response.url

'https://www.google.com/maps/place/Kari+dosa/@12.9136665,77.6026948,17z/data=!4m6!3m5!1s0x3bae1500439dae5d:0xff8fc22bcc73fc25!8m2!3d12.9136665!4d77.6052697!16s%2Fg%2F11vs35r34t?entry=tts'

In [6]:
# Regular expression to find latitude and longitude
pattern1 = r'@(-?\d+\.\d+),(-?\d+\.\d+)'
pattern2 = r'%40(-?\d+\.\d+),(-?\d+\.\d+)'

# Search for the pattern in the URL
match = re.search(pattern, api_response.url)

if match:
    latitude = match.group(1)
    longitude = match.group(2)
    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print("No latitude and longitude found.")

Latitude: 12.9136665, Longitude: 77.6026948


In [23]:
def extract_lat_long_from_dynamic_url(short_url):
    try:
        response = requests.get(short_url)
        final_url = response.url
        # Regular expressions to find latitude and longitude in different formats
        pattern1 = r'@(-?\d+\.\d+),(-?\d+\.\d+)'
        pattern2 = r'%40(-?\d+\.\d+),(-?\d+\.\d+)'
        # Search for the patterns in the URL
        match1 = re.search(pattern1, final_url)
        match2 = re.search(pattern2, final_url)
        if match1:
            latitude = match1.group(1)
            longitude = match1.group(2)
            return latitude, longitude
        elif match2:
            latitude = match2.group(1)
            longitude = match2.group(2)
            return latitude, longitude
        else:
            return None, None
    except requests.RequestException as e:
        print(f"Failed to fetch URL: {short_url}, error: {e}")
        return None, None

In [11]:
df.head()

,Title,Link,Description,Links,gmaps_links
0,Tasted This Unbelievably Tasty Root Soup! Muda...,https://www.youtube.com/watch?v=ERWIan4ZYr4,Watch the complete video at https://youtu.be/5...,['https://youtu.be/5s-CtOb09FY'],NaN
1,What We Ate At Erodu Amman Mess! Full Menu (al...,https://www.youtube.com/watch?v=EWWe60lHEKw,Watch full video at Watch at https://youtu.be/...,['https://youtu.be/Swvj1J9sJCk'],NaN
2,Found This Delicious Dosa Chicken Curry For 50...,https://www.youtube.com/watch?v=APg_flJyXHs,Watch the complete episode at Watch at https:/...,"['https://youtu.be/fUR70aUEMEw', 'https://maps...",['https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8']
3,How The Hoskote Biryani Tradition Began! #biry...,https://www.youtube.com/watch?v=niUlMJ8q6-M,The earliest customers for the Hoskote biryani...,['https://youtu.be/MsjFBRkP8Ew'],NaN
4,Delicious Tamil Nadu Meal At Sangeetha Chennai...,https://www.youtube.com/watch?v=CXlj9f8ZnX8,A delightful Tamil Nadu meal which in addition...,"['https://youtu.be/s2Tprj8T1uE', 'https://maps...",['https://maps.app.goo.gl/CzFaV8qf3YXkPczu9']


In [12]:
links = df[df["gmaps_links"].notnull()][["gmaps_links"]]

In [13]:
links

,gmaps_links
2,['https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8']
4,['https://maps.app.goo.gl/CzFaV8qf3YXkPczu9']
5,['https://goo.gl/maps/kqUtDU14PkpJV4gT6']
6,"['https://goo.gl/maps/48tg1bR4n3swbzGXA', 'htt..."
7,['https://maps.app.goo.gl/CzFaV8qf3YXkPczu9']
...,...
731,['https://maps.app.goo.gl/7BsHd7psfbHtcnoN6']
733,['https://maps.app.goo.gl/24YwTcG8qpEhM15FA']
736,['https://maps.app.goo.gl/3sKkHUEFtfT8ENzr7']
737,['https://maps.app.goo.gl/3sKkHUEFtfT8ENzr7']


In [14]:
def evaluate_list(string):
    return ast.literal_eval(string)
links["gmaps_links"] = links["gmaps_links"].apply(evaluate_list)

In [15]:
links.head()

,gmaps_links
2,[https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8]
4,[https://maps.app.goo.gl/CzFaV8qf3YXkPczu9]
5,[https://goo.gl/maps/kqUtDU14PkpJV4gT6]
6,"[https://goo.gl/maps/48tg1bR4n3swbzGXA, https:..."
7,[https://maps.app.goo.gl/CzFaV8qf3YXkPczu9]


In [16]:
all_links = [link for sublist in links['gmaps_links'] for link in sublist]
all_links

['https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8',
 'https://maps.app.goo.gl/CzFaV8qf3YXkPczu9',
 'https://goo.gl/maps/kqUtDU14PkpJV4gT6',
 'https://goo.gl/maps/48tg1bR4n3swbzGXA',
 'https://goo.gl/maps/wyn3Eai5PAu5f5wc8',
 'https://maps.app.goo.gl/CzFaV8qf3YXkPczu9',
 'https://maps.app.goo.gl/mvvkuqfdYxsCjD8t9',
 'https://maps.app.goo.gl/gGSmVCU9w5efBReT8',
 'https://maps.app.goo.gl/8Txm7yqFt3JX81469',
 'https://maps.app.goo.gl/mvvkuqfdYxsCjD8t9',
 'https://maps.app.goo.gl/uKGRoe6u3h8SNZ1J7',
 'https://maps.app.goo.gl/dcaGdjFmb2Apfnwz7',
 'https://maps.google.com/?q=11.785901,78.208954',
 'https://maps.google.com/?q=11.785557,78.208794',
 'https://maps.google.com/?q=11.784240,78.208786',
 'https://goo.gl/maps/zrCuARY8qXzBJM4q6',
 'https://maps.app.goo.gl/GmqsyKnwDCw78uZ37',
 'https://goo.gl/maps/bZat92ZaSkeRCVyJA',
 'https://goo.gl/maps/jX4YeivtBhozhtqX9',
 'https://maps.app.goo.gl/yTMfQKW3SrZSyFom8',
 'https://goo.gl/maps/n51JyEx7ToYTmUrD8',
 'https://maps.app.goo.gl/YohtprzBHRedfDiz5',
 

In [17]:
len(all_links)

703

In [22]:
import urllib3
import time
# Initialize the PoolManager
http = urllib3.PoolManager()

resulting_urls = []

for i in all_links:
    response = http.request("GET", i)
    time.sleep(1)
    print(response.geturl())
    resulting_urls.append(response.geturl())
    if len(resulting_urls) == 10:
        break

https://www.google.com/sorry/index?continue=https://www.google.com/maps/place/Kari%2Bdosa/%4012.9136665,77.6026948,17z/data%3D!4m6!3m5!1s0x3bae1500439dae5d:0xff8fc22bcc73fc25!8m2!3d12.9136665!4d77.6052697!16s%252Fg%252F11vs35r34t%3Fentry%3Dtts&q=EgSZIVVyGPTt77IGIjAHZqHloZU-TuU361-lwZw5Bd4Aqvl-635M4u17WgTFtix_Z2mCR_NvL9JIUqQ9j78yAXJaAUM
https://www.google.com/sorry/index?continue=https://www.google.com/maps/place/Shop%2BNo:7,%2BSangeetha%2BVeg%2BRestaurant,%2B1st%2BMain%2BRd,%2Bnear%2BReliance%2BDigital,%2BGandhi%2BNagar,%2BAdyar,%2BChennai,%2BTamil%2BNadu%2B600020/data%3D!4m2!3m1!1s0x3a526793f45a8e3f:0x9357bfba91e3ab6%3Futm_source%3Dmstt_1%26entry%3Dgps&q=EgSZIVVyGPbt77IGIjDSaTUpChXpGBJuq9bVnFURyLXd3r98M1E-4ZcvF0LBN7suUKBhgRqxijo1Medh_tgyAXJaAUM
https://www.google.com/sorry/index?continue=https://www.google.com/maps/place/Mani%2BBiryani%2BCentre,Hoskote/%4013.0752152,77.8024299,17z/data%3D!3m1!4b1!4m5!3m4!1s0x3bae058f1b6914d1:0x576c57269a649546!8m2!3d13.0752152!4d77.804624%3Fshorturl%3

In [21]:
resulting_urls

['https://www.google.com/sorry/index?continue=https://www.google.com/maps/place/Kari%2Bdosa/%4012.9136665,77.6026948,17z/data%3D!4m6!3m5!1s0x3bae1500439dae5d:0xff8fc22bcc73fc25!8m2!3d12.9136665!4d77.6052697!16s%252Fg%252F11vs35r34t%3Fentry%3Dtts&q=EgSZIVVyGPPz7bIGIjAdWbTkJn9jS1AVGdizzG7Nu0ZMxd59rTEwwwCVHCUNi0UGt4qnNUrCBRmytBlKL2UyAXJaAUM',
 'https://www.google.com/sorry/index?continue=https://www.google.com/maps/place/Shop%2BNo:7,%2BSangeetha%2BVeg%2BRestaurant,%2B1st%2BMain%2BRd,%2Bnear%2BReliance%2BDigital,%2BGandhi%2BNagar,%2BAdyar,%2BChennai,%2BTamil%2BNadu%2B600020/data%3D!4m2!3m1!1s0x3a526793f45a8e3f:0x9357bfba91e3ab6%3Futm_source%3Dmstt_1%26entry%3Dgps&q=EgSZIVVyGPTz7bIGIjC3eJ3WyaB1J6Xdn2LESnS9PhWsc16ZwKLimV9RBBxhYewMAvTDcSEwMc2lOxmiIbIyAXJaAUM',
 'https://www.google.com/sorry/index?continue=https://www.google.com/maps/place/Mani%2BBiryani%2BCentre,Hoskote/%4013.0752152,77.8024299,17z/data%3D!3m1!4b1!4m5!3m4!1s0x3bae058f1b6914d1:0x576c57269a649546!8m2!3d13.0752152!4d77.804624%3F

In [18]:
link_df = pd.DataFrame(all_links, columns=["link"])
link_df.head()

,link
0,https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8
1,https://maps.app.goo.gl/CzFaV8qf3YXkPczu9
2,https://goo.gl/maps/kqUtDU14PkpJV4gT6
3,https://goo.gl/maps/48tg1bR4n3swbzGXA
4,https://goo.gl/maps/wyn3Eai5PAu5f5wc8


In [24]:
link_df[['latitude', 'longitude']] = link_df['link'].apply(lambda x: pd.Series(extract_lat_long_from_dynamic_url(x)))

In [25]:
link_df

,link,latitude,longitude
0,https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8,12.9136665,77.6026948
1,https://maps.app.goo.gl/CzFaV8qf3YXkPczu9,None,None
2,https://goo.gl/maps/kqUtDU14PkpJV4gT6,13.0752152,77.8024299
3,https://goo.gl/maps/48tg1bR4n3swbzGXA,12.8918777,77.5887793
4,https://goo.gl/maps/wyn3Eai5PAu5f5wc8,12.9328672,77.5805927
...,...,...,...
698,https://maps.app.goo.gl/7BsHd7psfbHtcnoN6,None,None
699,https://maps.app.goo.gl/24YwTcG8qpEhM15FA,None,None
700,https://maps.app.goo.gl/3sKkHUEFtfT8ENzr7,None,None
701,https://maps.app.goo.gl/3sKkHUEFtfT8ENzr7,None,None


In [21]:
link_df[link_df["latitude"].isnull()]

,link,latitude,longitude
1,https://maps.app.goo.gl/CzFaV8qf3YXkPczu9,None,None
5,https://maps.app.goo.gl/CzFaV8qf3YXkPczu9,None,None
11,https://maps.app.goo.gl/dcaGdjFmb2Apfnwz7,None,None
12,"https://maps.google.com/?q=11.785901,78.208954",None,None
13,"https://maps.google.com/?q=11.785557,78.208794",None,None
...,...,...,...
698,https://maps.app.goo.gl/7BsHd7psfbHtcnoN6,None,None
699,https://maps.app.goo.gl/24YwTcG8qpEhM15FA,None,None
700,https://maps.app.goo.gl/3sKkHUEFtfT8ENzr7,None,None
701,https://maps.app.goo.gl/3sKkHUEFtfT8ENzr7,None,None


In [ ]:
df['latitude'], df['longitude'] = zip(*df['gmaps_links'].apply(extract_coordinates))